# Librerias

In [ ]:

from nltk.tokenize import regexp_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import nltk

import xml.etree.ElementTree as ET

from collections import defaultdict
import os
import re

from typing import List

# Configuración de parámetros

In [ ]:
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

In [ ]:
stemmer = SnowballStemmer("english")

pattern = r'''(?x)              
    (?:[A-Z]\.)+[A-Z]?          # abreviaturas, e.g. U.S.A.
  | [A-Za-z]+(?:-[A-Za-z]+)*    # palabras con guiones internos, e.g. non-euclidean
  | \$?\d+(?:\.\d+)?%?          # monedas, números, porcentajes
  | \.\.\.                      # puntos suspensivos
  | [][.,;"'?():_`-]            # puntuación como tokens separados
'''

# Proceso de preprocesamiento

In [78]:
def preprocess(text: str) -> List[str]:
    # Tokenización del texto a nivel de palabra usando expresión regular para inglés
    tokens = regexp_tokenize(text, pattern)
    # Normalización de los tokens
    tokens = [token.lower() for token in tokens]
    # Eliminación de palabras de parada
    tokens = [token for token in tokens if token not in stop_words]
    # Stemming de los tokens
    tokens = [stemmer.stem(token) for token in tokens]
    # Retornar tokens
    return tokens

In [79]:
print(preprocess("Se deben invertir $12.000 millones en D.I.A ..."))

['se', 'deben', 'invertir', '$12.000', 'millon', 'en', 'd.i.a', '...']


# Ingesta de documentos

In [80]:
# Ruta en la que se encuentran los documentos a procesar
path_docs = "../data/docs-raw-texts"

def load_documents(path_docs: str):
	# Diccionario para almacenar los documentos por id
	docs = {}
	# Se revisan únicamente los archivos relevantes
	for file_name in os.listdir(path_docs):
		if not file_name.endswith(".naf"):
			continue
		# Construccion del arbol de atributos del documento
		tree = ET.parse(os.path.join(path_docs, file_name))
		root = tree.getroot()
		# Id recuperado desde el atributo pupblicId
		doc_id = root.find("nafHeader/public").attrib["publicId"]
		# Titulo desde el atributo title
		title = root.find("nafHeader/fileDesc").attrib.get("title", "")
		# Contenido desde <raw>
		raw_element = root.find("raw")
		content = raw_element.text if raw_element is not None else ""
		# concatenar titulo + contenido para preprocesar
		tokens = preprocess(title +  " " + content)
		docs[doc_id] = tokens
	# Retornar los documentos cargados
	return docs

In [81]:
documents = load_documents(path_docs)
print(documents["d006"])

['eugenio', 'beltrami', 'non-euclidian', 'geometri', 'eugenio', 'beltrami', 'non-euclidian', 'geometri', '.', 'eugenio', 'beltrami', '(', '1835', '-', '1900', ')', '.', 'novemb', '16', ',', '1835', ',', 'italian', 'mathematician', 'eugenio', 'beltrami', 'born', '.', 'notabl', 'work', 'concern', 'differenti', 'geometri', 'mathemat', 'physic', '.', 'work', 'note', 'especi', 'clariti', 'exposit', '.', 'first', 'prove', 'consist', 'non-euclidean', 'geometri', 'model', 'surfac', 'constant', 'curvatur', ',', 'pseudospher', '.', 'eugenio', 'beltrami', 'born', 'cremona', 'lombardi', ',', 'part', 'austrian', 'empir', ',', 'part', 'itali', '.', 'son', 'artist', 'paint', 'miniatur', ',', 'young', 'eugenio', 'certain', 'inherit', 'artist', 'talent', 'famili', ',', 'case', 'addit', 'mathemat', 'talent', 'would', 'acquir', ',', 'musicrath', 'paint', 'becam', 'import', 'life', '.', 'began', 'studi', 'mathemat', 'univers', 'pavia', '1853', ',', 'expel', 'ghislieri', 'colleg', '1856', 'due', 'polit', '

# Construcción de índice invertido

In [82]:
def build_inverted_index(docs):
    # Construcción de la estructura de indice invertido
    inverted = defaultdict(set)
    # Iteración sobre los doumentos cargados
    for doc_id, tokens in docs.items():
        # Iteración sobre los tokens de los documentos
        for token in tokens:
            # Población del índice
            inverted[token].add(doc_id)
    # Convertir sets a listas ordenadas
    for token in inverted:
        inverted[token] = {"df" : len(inverted[token]), "postings" : sorted(inverted[token])}
    # Se retorna el índice construido
    return inverted

In [83]:
index = build_inverted_index(documents)

# Calculo de consultad mediante algoritmo de mezcla

In [84]:
def merge_and(postings_1, postings_2):
	# Inicialización de los punteros
	i, j = 0, 0
	result = []
	# Busqueda mediante la estrategia de dos punteros
	while i < len(postings_1) and j < len(postings_2):
		# Si los valores son iguales, se cumple el and
		if postings_1[i] == postings_2[j]:
			result.append(postings_1[i])
			i += 1
			j += 1
		# Se avanza el puntero que apunta al documento con menor doc_id
		elif postings_1[i] < postings_2[j]:
			i += 1
		else:
			j += 1
	return result

In [85]:
def merge_not(postings, all_docs):
	# Inicialización de los punteros
	result = []
	i, j = 0, 0
	# Busqueda mediante la estrategia de dos punteros
	while i < len(all_docs) and j < len(postings):
		# Si el termino esta tanto en el posting como en la lista de todos los documentos,
		# no se cumple el not
		if all_docs[i] == postings[j]:
			i += 1
			j += 1
		# Se cumple el not y se agrega
		elif all_docs[i] < postings[j]:
			result.append(all_docs[i])
			i += 1
		else:
			j += 1
	# Se agregan todos los no evaluados porque cumplen el not
	result.extend(all_docs[i:])
	return result

In [86]:
def boolean_query(query_tokens, index, all_docs):
    stack = []
    for token in query_tokens:
        if token == "AND":
            postings_1 = stack.pop()
            postings_2 = stack.pop()
            stack.append(merge_and(postings_1, postings_2))
        elif token == "NOT":
            postings = stack.pop()
            stack.append(merge_not(postings, all_docs))
        else:
            postings = index.get(token, {"postings": []})["postings"]
            stack.append(postings)
    return stack.pop() if stack else []

In [90]:
def read_query(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    return root.find("raw").text.strip()

In [91]:
print(read_query("../data/queries-raw-texts/wes2015.q01.naf"))

Fabrication of music instruments


In [ ]:
## def process_queries(path_queries, index, all_docs, output_file)